In [ ]:
# default_exp notebook

# Notebook

> Functions to use `py2gift` from a Jupyter notebook.

In [ ]:
# export
import argparse
import json

from IPython.core.magic import Magics, magics_class, line_magic, cell_magic, line_cell_magic

import py2gift.input_file

## Convenience functions

A class to act as a *container*. This will be useful to simulate a *Python* module.

In [ ]:
# export
class ClassesContainer:
    
    def add(cls, class_to_add):
        
        setattr(cls, class_to_add.__name__, class_to_add)

In [ ]:
classes_container = ClassesContainer()

In [ ]:
class DummyClass:
    
    n = 3

In [ ]:
classes_container.add(DummyClass)

In [ ]:
classes_container.DummyClass

In [ ]:
classes_container.DummyClass.n

## Magics

In [ ]:
# export
# The class MUST call this class decorator at creation time
@magics_class
class MyMagics(Magics):
    
    statement_key = 'statement'
    feedback_key = 'feedback'
    
    def __init__(self, shell=None,  **kwargs):
        
        super().__init__(shell=shell, **kwargs)
        
        
        self.location_parser = argparse.ArgumentParser(description='Specification')
        self.location_parser.add_argument('settings', help='Settings object')
        self.location_parser.add_argument('-c', '--cls', default=None, help='class')
        
        # the name of a category can contain spaces; notice that this will yield a *list* rather than a string
        self.location_parser.add_argument('-C', '--category', default=None, nargs='+', help='category')
    
    def process(self, line: str, cell: str, key: str):
        
        line_arguments = self.location_parser.parse_args(line.split())
        
#         print(f'line_arguments.category={line_arguments.category}')
        
        if line_arguments.category:
            
            category = json.loads(' '.join(line_arguments.category))
        
        else:
            
            category = None
        
        settings = self.shell.user_ns[line_arguments.settings]
        
        if category is None:
            
            category = settings.store['categories'][-1]['name']
            
        
        cls = line_arguments.cls
        
        if cls is None:
            
            cls = settings.locate(category_name=category)['classes'][-1]['name']
        

#         print(f'category={category}, class={cls}')
        
        settings.locate(category_name=category, class_name=cls)[key] = cell

    @cell_magic
    def statement(self, line, cell):

        self.process(line, cell, self.statement_key)
        
        return f'statement recorded'
    
    @cell_magic
    def feedback(self, line, cell):
        
        self.process(line, cell, self.feedback_key)
        
        return f'feedback recorded'

## Parser

The class can be instantiated directly,

In [ ]:
m = MyMagics()

to test the parser,

In [ ]:
m.location_parser.parse_args('settings -c myclass -C mycategory'.split())

When no category nor class are passed:

In [ ]:
m.location_parser.parse_args('settings'.split())

Parsed category is always a list

In [ ]:
m.location_parser.parse_args('settings -c myc -C cat 1'.split())

`json` should be used to pass the category. Then, within the *magic*, the `category` list is joined (with a space) and parsed back using `json` to whatever type it originally had.

In [ ]:
category = 'foo'
line = m.location_parser.parse_args(f'settings -c myc -C {json.dumps(category)}'.split())
json.loads(' '.join(line.category))

In [ ]:
category = 'foo foo'
line = m.location_parser.parse_args(f'settings -c myc -C {json.dumps(category)}'.split())
json.loads(' '.join(line.category))

In [ ]:
category = ['cat', 'subcat']
line = m.location_parser.parse_args(f'settings -c myc -C {json.dumps(category)}'.split())
json.loads(' '.join(line.category))

Notice that a `json`-*dumped* string is not equal to the string,

In [ ]:
json.dumps('Category 1') == 'Category 1'

meaning that, if in a variable, the category must **always** be dumped

## Actual use

*Magics* must be somehow registered

In [ ]:
# export

get_ipython().register_magics(MyMagics)

In [ ]:
# %lmagic -n foo

In [ ]:
settings = py2gift.input_file.Settings()
category_name = settings.add_category('Category 1')
settings.add_or_update_class(category_name=category_name, class_name='question name', question_base_name='base name', n_instances=2)

In [ ]:
settings.to_dict()

In [ ]:
%%statement settings
Consider

bla bla

In [ ]:
settings

In [ ]:
cls = 'ClassA'
category = 'Category 1'
settings.add_or_update_class(category_name=category, class_name=cls, question_base_name='base name', n_instances=2)

In [ ]:
settings

In [ ]:
%%statement settings -c {cls} -C {json.dumps(category)}
more blah

In [ ]:
cls = 'cls'
category = 'foo foo'

settings = py2gift.input_file.Settings()
category_name = settings.add_category(category)
settings.add_or_update_class(category_name=category_name, class_name=cls, question_base_name='base name', n_instances=2)

In [ ]:
%%statement settings -C {json.dumps(category)} -c {cls}
more

In [ ]:
cls = 'cls'
category = 'wap wap'
base_category = 'oh'

settings = py2gift.input_file.Settings()
category_name = settings.add_category(category_name=category, base_category=base_category)
settings.add_or_update_class(category_name=category_name, class_name=cls, question_base_name='base name', n_instances=2)

In [ ]:
category_name

In [ ]:
settings

In [ ]:
%%feedback  settings -C {json.dumps(category_name)} -c {cls}
We must...

In [ ]:
settings

In [ ]:
# hide
import nbdev.export
nbdev.export.notebook2script('60_notebook.ipynb')